In [12]:
from treelib import Node, Tree

#create tree object from an annotation file
#Sample Annotation:
#
#
# Q0{What is the way things are?}
# >Q1{###Q_1###}
# >>A1.1{###Answer_1’###}
# >>Q2{###Question_2###}
# >>>A2.1{###Answer_2’###}
# >>>A2.2{###Answer_2’’###}
# >>Q3{###Question_3###}
# >>>A3{###Answer_3###}
# >>>Q4{###Question_4###}
# >>>>A4{###Answer_4###}
# >>A1.2{###Answer_1’’###}
#
def createTreeFromFile(filename):
    tree = Tree()
    with open (filename, "r") as myfile:
        data=myfile.readlines()

    level_dict = dict()
    node_no=0
    node_id=getID(data[0])
    tree.create_node(getQuestion(data[0]), node_id, data=tree_node(getType(data[0]), node_no))
    level_dict[0] = node_id
    for node in data[1:]:
        node_no = node_no+1
        level = getLevel(node)
        text = getQuestion(node)
        node_type = getType(node)
        node_id=getID(node)
        tree.create_node(text,node_id,parent=level_dict[level-1], data=tree_node(node_type, node_no))
        level_dict[level] = node_id

    tree.show(key=lambda x: x.data.node_no)

    leafNodeIDs = [leaf.identifier for leaf in tree.leaves()]
    assert all(leafNode.startswith("A") for leafNode in leafNodeIDs), "all leaf nodes should be Answers and no Questions: "+ str(leafNodeIDs)
    internalNodeIDs = []
    for node in tree.nodes.keys():
        if not tree.get_node(node).is_leaf():
            internalNodeIDs.append(node)
    assert all(internalNode.startswith("Q") for internalNode in internalNodeIDs), "all internal nodes should be Questions and no Answers: "+ str(internalNodeIDs)

    return tree

In [1]:
import re
def getQuestion(string):
    p = re.compile(".*{(.*)}.*")
    return p.match(string).group(1)

def getLevel(string):
    p = re.compile(">*")
    return p.match(string).span()[1]-p.match(string).span()[0]

def getID(string):
    p = re.compile(">*(.*){")
    return p.match(string).group(1)

def getType(string):
    p = re.compile(">*(A|Q*)")
    return p.match(string).group(1)

In [2]:
class tree_node(object):
    def __init__(self, node_type, node_no):
        self.node_type = node_type
        self.node_no = node_no

In [13]:
createTreeFromFile("sample.txt")

What is the way things are?
└── ###Q_1###
    ├── ###Answer_1’###
    ├── ###Question_2###
    │   ├── ###Answer_2’###
    │   └── ###Answer_2’’###
    ├── ###Question_3###
    │   ├── ###Answer_3###
    │   └── ###Question_4###
    │       └── ###Answer_4###
    └── ###Answer_1’’###

